In [1]:
import numpy as np
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28416\2936652086.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


conclusion: 

Feature Selection: CCI, ADX, Chaikin A/D
Model Selection: best using SVC with poly kernel
Hyperparameter Tuning: d = 2 c = 0.3

Report

1. SVM -> stock price -> goblok tebak all up or down
2. Data collection -> jelasin features dari alphavantage + feature buatan sendiri (short SMA / long SMA ratio)
3. Model Selection -> analyze correlation  bilkang not linearly separable ...
4. Feature Selection
5. Hyperparameter tuning

stevan: 
- feature buatan sendiri (short SMA / long SMA ratio)
- training data selection
- improve acc to >70%

In [2]:
'''
CONST
'''
FILEPATH_STOCK_PRICE = "./tr_eikon_eod_data.csv"
STOCK_NAMES = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']

In [15]:
''' 
UTILS
'''
def sequential_train_test_split(data, ratio = 0.8):
    df = pd.DataFrame(data.dropna()).to_numpy()
    split_index = int(len(df) * ratio)
    return df[:split_index], df

def calculate_return(prediction, data) :
    data = data
    prediction = pd.DataFrame(prediction)
    temp = pd.DataFrame(data)
    temp['log_return'] = np.log(temp / temp.shift(1))
    temp['returne'] = prediction.shift(1) * temp['log_return'].values.reshape(-1, 1)
    temp.dropna(inplace = True) 

    return np.exp(temp[['log_return', 'returne']].sum())


In [5]:
'''
BENCHMARK
'''

def benchmark_accuracy(prediction, test):
    prediction = pd.DataFrame(prediction)
    test = pd.DataFrame(test)

    return (prediction == test).sum() / len(prediction)


In [6]:
''' 
MODEL
'''

def predict_position(train, test):
    # Train Hidden Markov Model
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(train.reshape(-1, 1))

    # Predict hidden states for test data
    hidden_states = model.predict(test.reshape(-1, 1))

    # Define a function to map hidden states to rise or fall
    def map_to_rise_or_fall(state):
        return 1 if state == 0 else -1

    # Map hidden states to rise or fall
    predicted_rise_fall = np.array(list(map(map_to_rise_or_fall, hidden_states)))

    return predicted_rise_fall

In [7]:
apple_data = pd.read_csv('apple_output.csv', index_col = 0, parse_dates = True).dropna()
apple_data['SMA_EMA_ratio'] = apple_data['SMA'] / apple_data['EMA']
SMA_short = 7
SMA_long = 20
apple_data['SMA_Short'] = apple_data['AAPL.O'].rolling(SMA_short).mean()
apple_data['SMA_Long'] = apple_data['AAPL.O'].rolling(SMA_long).mean()
apple_data['SMA_Long_Short'] = apple_data['SMA_Short'] / apple_data['SMA_Long']
apple_data['data'] = apple_data['AAPL.O']
apple_data['direction'] = np.where(apple_data['AAPL.O'] <= apple_data['AAPL.O'].shift(-1), 1, -1)

apple_data

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,30.572827,78.7484,82.8722,75.0702,38.0954,96.8893,42.8571,100.0000,8.816356e+10,5.524930e+10,6.9552,6.9908,0.994908,NaN,NaN,NaN,30.572827,-1
2010-06-01,30.138541,89.5060,82.1665,57.5986,35.3551,151.4643,57.1429,100.0000,8.681483e+10,6.043320e+10,7.5207,7.6163,0.987448,NaN,NaN,NaN,30.138541,-1
2010-07-01,30.082827,11.7287,14.7644,38.9499,24.7259,-152.8759,100.0000,42.8571,8.390127e+10,5.897666e+10,8.1018,7.9595,1.017878,NaN,NaN,NaN,30.082827,-1
2010-11-01,30.015684,26.0805,26.1764,50.8046,19.7333,-103.8405,0.0000,28.5714,9.091934e+10,6.372971e+10,9.2871,9.2333,1.005827,NaN,NaN,NaN,30.015684,-1
2010-12-01,29.674256,71.5220,76.9725,59.2894,18.8722,97.8421,35.7143,0.0000,9.076772e+10,6.438059e+10,9.3769,9.4407,0.993242,NaN,NaN,NaN,29.674256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,182.170000,24.7765,28.7326,33.5305,28.2742,-132.0057,100.0000,14.2857,9.108196e+10,7.277260e+10,44.7743,44.5714,1.004552,186.045714,188.8650,0.985072,182.170000,1
2018-06-26,184.430000,30.9769,28.9171,32.5318,28.2014,-70.9622,92.8571,7.1429,9.118024e+10,7.276743e+10,44.5744,44.3849,1.004269,185.415714,188.6790,0.982705,184.430000,-1
2018-06-27,184.160000,37.4891,31.0808,42.8627,27.4502,-43.5508,85.7143,0.0000,9.107910e+10,7.267438e+10,44.4033,44.3292,1.001672,184.761429,188.4580,0.980385,184.160000,1


In [16]:
scaler = MinMaxScaler()

# Normalize the selected columns
normalized_columns = scaler.fit_transform(apple_data.iloc[:, :-2])

# Create a new DataFrame with the normalized values
normalized_df = pd.DataFrame(normalized_columns, columns=apple_data.columns[:-2])

# Display the normalized DataFrame
apple_data.iloc[:, :-2] = normalized_df
apple_data


,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio,SMA_Short,SMA_Long,SMA_Long_Short,data,direction
Date,,,,,,,,,,,,,,,,,,
2010-04-01,0.018826,0.797082,0.851941,0.769280,0.413634,0.658869,0.428571,1.000000,0.466485,0.197673,0.021918,0.021476,0.436146,NaN,NaN,NaN,30.572827,-1
2010-06-01,0.016218,0.907128,0.844247,0.536156,0.374250,0.748379,0.571429,1.000000,0.417465,0.386577,0.033742,0.034488,0.269045,NaN,NaN,NaN,30.138541,-1
2010-07-01,0.015884,0.111497,0.109400,0.287327,0.221484,0.249224,1.000000,0.428571,0.311572,0.333500,0.045891,0.041627,0.950701,NaN,NaN,NaN,30.082827,-1
2010-11-01,0.015480,0.258310,0.233819,0.445504,0.149730,0.329648,0.000000,0.285714,0.566643,0.506703,0.070674,0.068125,0.680743,NaN,NaN,NaN,30.015684,-1
2010-12-01,0.013430,0.723159,0.787620,0.558717,0.137354,0.660432,0.357143,0.000000,0.561133,0.530421,0.072551,0.072439,0.398836,NaN,NaN,NaN,29.674256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.929087,0.244971,0.261688,0.215016,0.272481,0.283454,1.000000,0.142857,0.572554,0.836229,0.812646,0.803231,0.652194,0.962688,0.996360,0.435173,182.170000,1
2018-06-26,0.942657,0.308398,0.263699,0.201690,0.271435,0.383573,0.928571,0.071429,0.576126,0.836040,0.808467,0.799351,0.645859,0.958849,0.995204,0.420941,184.430000,-1
2018-06-27,0.941036,0.375016,0.287289,0.339535,0.260639,0.428531,0.857143,0.000000,0.572450,0.832650,0.804889,0.798193,0.587664,0.954862,0.993830,0.407000,184.160000,1


In [9]:
# apple_data = apple_data[['SlowK', 'RSI', 'CCI', 'Aroon Down', 'OBV', 'direction']]

In [17]:
train_data, test_data = sequential_train_test_split(apple_data)
print(test_data)
X_train, y_train, X_test, y_test = train_data[:, :-1], train_data[:, -1], test_data[:, :-1], test_data[:, -1]

[[ 1.76445609e-03  3.46766521e-01  3.87230206e-01 ...  3.41359070e-01
   2.77314008e+01  1.00000000e+00]
 [ 3.54006332e-03  9.01567586e-01  7.73191369e-01 ...  3.25815501e-01
   2.80271148e+01  1.00000000e+00]
 [ 5.66736088e-03  4.30623781e-01  2.79140278e-01 ...  3.04549806e-01
   2.83814002e+01  1.00000000e+00]
 ...
 [ 9.41036056e-01  3.75015600e-01  2.87288752e-01 ...  4.07000417e-01
   1.84160000e+02  1.00000000e+00]
 [ 9.49082053e-01  5.58899048e-01  3.98814034e-01 ...  4.09888669e-01
   1.85500000e+02 -1.00000000e+00]
 [ 9.46740307e-01  6.21346749e-01  5.09991529e-01 ...  4.07290627e-01
   1.85110000e+02 -1.00000000e+00]]


In [11]:
X_train

array([[1.76445609e-03, 3.46766521e-01, 3.87230206e-01, ...,
        4.80943814e-03, 3.41359070e-01, 2.77314008e+01],
       [3.54006332e-03, 9.01567586e-01, 7.73191369e-01, ...,
        4.01821057e-03, 3.25815501e-01, 2.80271148e+01],
       [5.66736088e-03, 4.30623781e-01, 2.79140278e-01, ...,
        3.47207707e-03, 3.04549806e-01, 2.83814002e+01],
       ...,
       [5.38496058e-01, 6.97975764e-01, 6.92562474e-01, ...,
        5.34636510e-01, 6.52213850e-01, 1.17120000e+02],
       [5.38135789e-01, 7.39940709e-01, 7.13861454e-01, ...,
        5.35718122e-01, 6.50464560e-01, 1.17060000e+02],
       [5.35373731e-01, 7.16791093e-01, 7.22948627e-01, ...,
        5.36659870e-01, 6.44616475e-01, 1.16600000e+02]])

In [12]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', gamma=0.7, C=1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
benchmark_accuracy(pred, y_test)

0    0.502674
dtype: float64

In [23]:
best_acc = 0
best_g = -1
best_c = -1
best_return = 0
bestgg = -1
bestcc = -1
log_return = 0  
trades = 0

selected_features = [1, 5, 11, 13]
dat = X_test[:,-1]

for g in range(1, 11, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', gamma=g / 10, C=c / 10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)
        #perfect_return = calculate_return(y_test, dat)
        log_return = temp.values[0]
        if acc[0] > best_acc:
            best_acc = acc[0]
            best_g = g
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestgg = g
            bestcc = c
            trades = (pd.DataFrame(pred).diff().dropna() != 0).sum()

print(f"best acc: {best_acc}, g: {best_g}, c: {best_c}")
print(f"best ret: {best_return}, g: {bestgg}, c: {bestcc}")
print(trades)

best acc: 0.534796573875803, g: 6, c: 34
best ret: 8.911289059205469, g: 6, c: 34
0    27
dtype: int64


In [20]:
print(log_return)

6.675104552706013


In [107]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]
dat = X_test[:,-1]

for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='sigmoid', C= c/10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.5423280423280423, c: 5
best ret: 2.328348097077286, c: 6


In [133]:
best_acc = 0
best_d = -1
best_c = -1
best_return = 0
bestdd = -1
bestcc = -1

selected_features = [1, 5, 11, 13]
dat = X_test[:,-1]

for d in range(1, 5, 1):
    for c in range(1, 51, 1):
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree=d, C=c/10)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp = calculate_return(pred, dat)

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_d = d
            best_c = c
        if temp.values[1] > best_return :
            best_return = temp.values[1]
            bestdd = d
            bestcc = c
    
print(f"best acc: {best_acc}, d: {best_d}, c: {best_c}")
print(f"best ret: {best_return}, d : {bestdd}, c: {bestcc}")

best acc: 0.5909090909090909, d: 2, c: 3
best ret: 2.747154336066217, d : 2, c: 3


In [109]:
best_acc = 0
best_c = -1
best_return = 0
bestcc = -1

selected_features = [4, 5, 9]

# for d in range(1, 5, 1):
for c in range(1, 101, 1):
    curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
    curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
    model = SVC(kernel='poly', degree=2, C = c / 10)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp = calculate_return(pred, dat)

    if acc[0] > best_acc:
        best_acc = acc[0]
        best_c = c
    if temp.values[1] > best_return :
        best_return = temp.values[1]
        bestcc = c
    
print(f"best acc: {best_acc}, c: {best_c}")
print(f"best ret: {best_return}, c: {bestcc}")

best acc: 0.6216931216931217, c: 3
best ret: 1.6194088342545465, c: 3


In [76]:
# Feature Engineering

# Feature Selection

In [77]:
X = apple_data.dropna().iloc[:, :-1]
y = apple_data.dropna().iloc[:, -1]

In [78]:
X

,AAPL.O,SlowK,SlowD,RSI,ADX,CCI,Aroon Down,Aroon Up,OBV,Chaikin A/D,SMA,EMA,SMA_EMA_ratio
Date,,,,,,,,,,,,,
2010-04-01,0.018837,0.797082,0.851941,0.769280,0.413634,0.658869,0.428571,1.000000,0.466485,0.197673,0.021918,0.021476,0.436146
2010-06-01,0.016229,0.907128,0.844247,0.536156,0.374250,0.748379,0.571429,1.000000,0.417465,0.386577,0.033742,0.034488,0.269045
2010-07-01,0.015895,0.111497,0.109400,0.287327,0.221484,0.249224,1.000000,0.428571,0.311572,0.333500,0.045891,0.041627,0.950701
2010-11-01,0.015491,0.258310,0.233819,0.445504,0.149730,0.329648,0.000000,0.285714,0.566643,0.506703,0.070674,0.068125,0.680743
2010-12-01,0.013441,0.723159,0.787620,0.558717,0.137354,0.660432,0.357143,0.000000,0.561133,0.530421,0.072551,0.072439,0.398836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-25,0.929088,0.244971,0.261688,0.215016,0.272481,0.283454,1.000000,0.142857,0.572554,0.836229,0.812646,0.803231,0.652194
2018-06-26,0.942658,0.308398,0.263699,0.201690,0.271435,0.383573,0.928571,0.071429,0.576126,0.836040,0.808467,0.799351,0.645859
2018-06-27,0.941037,0.375016,0.287289,0.339535,0.260639,0.428531,0.857143,0.000000,0.572450,0.832650,0.804889,0.798193,0.587664


In [79]:
y

Date
2010-04-01    1
2010-06-01   -1
2010-07-01    1
2010-11-01   -1
2010-12-01    1
             ..
2018-06-25    1
2018-06-26   -1
2018-06-27    1
2018-06-28   -1
2018-06-29   -1
Name: direction, Length: 1887, dtype: int32

In [126]:
best_acc = 0
best_ret = 0

dat = X_test[:,-1]
for i in range(len(apple_data.columns) - 1):
    print(X_train[:,i])
    curr_X_train = X_train[:,i].reshape(-1, 1)
    curr_X_test = X_test[:,i].reshape(-1, 1)
    model = SVC(kernel='rbf', gamma= 1.0, C= 1.0)
    model.fit(curr_X_train, y_train)
    pred = model.predict(curr_X_test)
    up_percentage = ((pred > 0).sum() / len(pred))
    acc = benchmark_accuracy(pred, y_test)
    temp =calculate_return(pred, dat).values[1]
    if temp > best_ret:
        best_ret = temp
    
    print(f"feature {i}: acc: {round(acc[0], 3)} up: {round(up_percentage, 3)}")
print(best_ret)

[0.00176446 0.00354006 0.00566736 ... 0.53849606 0.53813579 0.53537373]
feature 0: acc: 0.521 up: 1.0
[0.34676652 0.90156759 0.43062378 ... 0.69797576 0.73994071 0.71679109]
feature 1: acc: 0.521 up: 1.0
[0.38723021 0.77319137 0.27914028 ... 0.69256247 0.71386145 0.72294863]
feature 2: acc: 0.521 up: 1.0
[0.47558165 0.4872501  0.53657915 ... 0.60691808 0.56825275 0.56303297]
feature 3: acc: 0.521 up: 1.0
[0.09337628 0.28172702 0.13789996 ... 0.28586191 0.25475756 0.22751647]
feature 4: acc: 0.521 up: 1.0
[0.62333069 0.67181331 0.5712516  ... 0.48528352 0.5320104  0.4213444 ]
feature 5: acc: 0.521 up: 1.0
[0.285714 0.714286 0.       ... 0.071429 0.       0.      ]
feature 6: acc: 0.521 up: 1.0
[0.785714 0.214286 0.214286 ... 0.571429 0.5      0.428571]
feature 7: acc: 0.521 up: 1.0
[0.27013045 0.29654949 0.58241438 ... 0.52167255 0.51816515 0.51479341]
feature 8: acc: 0.521 up: 1.0
[0.36396515 0.36676005 0.51428661 ... 0.70766277 0.70903593 0.70908959]
feature 9: acc: 0.521 up: 1.0
[0.0

In [127]:
best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(len(apple_data.columns) - 1):
    for j in range(len(apple_data.columns) - 1):
        selected_features = [i, j]
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

best acc: 0.56951871657754 features: [5, 9]
best ret: 2.1594801331865385 features: [2, 9]


In [129]:
print(apple_data.columns)

Index(['AAPL.O', 'SlowK', 'SlowD', 'RSI', 'ADX', 'CCI', 'Aroon Down',
       'Aroon Up', 'OBV', 'Chaikin A/D', 'SMA', 'EMA', 'SMA_EMA_ratio',
       'SMA_Short', 'SMA_Long', 'SMA_Long_Short', 'data', 'direction'],
      dtype='object')


In [132]:
import itertools

all_features = list(range(len(apple_data.columns) - 2))

best_acc = 0
best_selected_features = []
best_ret = 0
best_selected_featuresa = []

for i in range(1, len(apple_data.columns) - 2):
    print(i)
    possible_selected_features = list(itertools.combinations(all_features, i))
    for selected_features in possible_selected_features:
        print(selected_features)
        curr_X_train = X_train[:, selected_features].reshape(-1, len(selected_features))
        curr_X_test = X_test[:, selected_features].reshape(-1, len(selected_features))
        model = SVC(kernel='poly', degree = 2, C= 0.3)
        model.fit(curr_X_train, y_train)
        pred = model.predict(curr_X_test)
        up_percentage = ((pred > 0).sum() / len(pred))
        acc = benchmark_accuracy(pred, y_test)
        temp =calculate_return(pred, dat).values[1]
        if temp > best_ret:
            best_ret = temp
            best_selected_featuresa = selected_features

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_selected_features = selected_features
print(f"best acc: {best_acc} features: {best_selected_features}")
print(f"best ret: {best_ret} features: {best_selected_featuresa}")

1
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
2
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(2, 13)
(2, 14)
(2, 15)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(3, 12)
(3, 13)
(3, 14)
(3, 15)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(4, 13)
(4, 14)
(4, 15)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(6, 13)
(6, 14)
(6, 15)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(7, 15)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
(8, 15)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(9, 15)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(12, 13)
(12, 14)
(12, 15)
(13, 14

array([[0.01883667, 0.79708231],
       [0.01622904, 0.9071284 ],
       [0.01589451, 0.11149665],
       ...,
       [0.53766059, 0.68329218],
       [0.54156345, 0.72807436],
       [0.5410831 , 0.67998392]])